<img align="left" src=https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=150 style="padding: 5px">
<img align="left" src=https://noirlab.edu/public/media/archives/images/screen640/ann22016a.jpg width=200 style="padding: 5px">
<br>

<b>What's That Streak?</b><br>
A Rubin x SatHub notebook by Meredith Rawls<br>
Last verified to run: June 17, 2025<br>
LSST Science Pipelines version: w_2025_24<br>

### Edit this top cell with the Rubin visit of interest, and run the notebook

The parameter `obsinfo` is a visit ID, which may typically constructed as date_obs + 00 + seq_num.

Only one cell below uses the Butler, which is instantiated via `repo`, `collections`, and `instrument`. Use a default collection that includes the desired visit, and ideally a `preliminary_visit_summary` data product (this is used to draw the camera footprint on the sky). The Butler is only used to retrieve basic information about where in the sky the telescope was pointing, and when, which is pipeline-agnostic.

In [ ]:
obsinfo = 2025091500319

# LSSTCam
repo = 'main'
collections = ['LSSTCam/raw/all', 'LSSTCam/calib/unbounded']
#['LSSTCam/runs/nightlyValidation']
instrument = 'LSSTCam'

# LSSTComCam
#repo = '/repo/dp1'
#collections = ['LSSTComCam/DP1']
#instrument = 'LSSTComCam'

### You shouldn't need to change anything below, it should "just work"

The SatChecker query includes a generous 3 degree radius FOV and a generous duration 3x the exposure time (centered on the exposure midpoint).

To learn more about SatChecker, please visit https://satchecker.readthedocs.io

In [ ]:
%matplotlib widget

In [ ]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, EarthLocation
import matplotlib.pyplot as plt
import numpy as np
import requests
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.patches import Polygon
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import lsst.daf.butler as dafButler
from lsst.obs.lsst import LsstCam

In [ ]:
fov_radius = 3  # FOV query radius, in degrees
# Note LSSTComCam is about 0.67 deg across on the sky ... LSSTCam FOV has a radius of 1.76 deg

In [ ]:
location=EarthLocation.of_site('Rubin')
latitude = location.lat.value  # deg
longitude = location.lon.value  # deg
elevation = location.height.value  # meters

In [ ]:
butler = dafButler.Butler(repo, collections=collections, instrument=instrument)
camera = LsstCam().getCamera()
detectors = [det.getId() for det in camera]
visit_record = butler.query_dimension_records("visit", instrument="LSSTCam", visit=obsinfo)[0]
cornersList = []
try:
    pvs = butler.get('preliminary_visit_summary', visit=obsinfo)

except dafButler.DatasetNotFoundError:
    print('No preliminary_visit_summary found, using summaryStats instead (slower)')
    for detector in detectors:
        try:
            summaryStats = butler.get('preliminary_visit_image.summaryStats', visit=obsinfo, detector=detector)
        except dafButler.DatasetNotFoundError:
            print(f'No summaryStats available for detector {detector}')
            cornersList.append([(None, None), (None, None), (None, None), (None, None)])
            continue
        else:
            cornersList.append(list(zip(summaryStats.raCorners, summaryStats.decCorners)))
else:
    for row in pvs:
        cornersList.append(list(zip(row["raCorners"], row["decCorners"])))

In [ ]:
exp_time = visit_record.exposure_time*u.s
exp_begin = visit_record.timespan.begin.utc

# the query time window will go from T0 - Texp to T0 + 2*Texp, for a total duration of 3*Texp
#start_time_jd = (exp_begin - exp_time).jd
#duration = (exp_time * 3).value
start_time_jd = exp_begin.jd
duration = exp_time.value

In [ ]:
coord = SkyCoord(alt=90*u.deg - visit_record.zenith_angle*u.deg,
                az=visit_record.azimuth*u.deg,
                obstime=exp_begin + (exp_time/2),
                location=EarthLocation.of_site('Rubin'),
                frame = 'altaz')
ra_center = coord.icrs.ra.value  # deg
dec_center = coord.icrs.dec.value  # deg

fov_radius = 1.7

In [ ]:
# Make the SatChecker API request
url_string = f"https://satchecker.cps.iau.org/fov/satellite-passes/?latitude={latitude}&longitude={longitude}&elevation={elevation}&start_time_jd={start_time_jd}&duration={duration}&ra={ra_center}&dec={dec_center}&fov_radius={fov_radius}&group_by=satellite"
#url_string = f"https://dev.satchecker.cps.iau.noirlab.edu/fov/satellite-passes/?latitude={latitude}&longitude={longitude}&elevation={elevation}&start_time_jd={start_time_jd}&duration={duration}&ra={ra_center}&dec={dec_center}&fov_radius={fov_radius}&group_by=satellite"
print(url_string)
response = requests.get(url_string, timeout=240)
data = response.json()


In [ ]:
data = {"data":{"satellites":{"DELTA 1 DEB (10842)":{"name":"DELTA 1 DEB","norad_id":10842,"positions":[{"altitude":77.30133067,"angle":2.91959102,"azimuth":174.29794819,"date_time":"2025-09-10 08:24:23 UTC","dec":-42.97032356,"julian_date":2460928.8502734,"ra":46.45061189,"range_km":1508.1870952,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":77.56685842,"angle":2.65123349,"azimuth":174.57076753,"date_time":"2025-09-10 08:24:24 UTC","dec":-42.71351227,"julian_date":2460928.85028497,"ra":46.33220818,"range_km":1507.11737641,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":77.83242413,"angle":2.38315842,"azimuth":174.85580658,"date_time":"2025-09-10 08:24:25 UTC","dec":-42.45626193,"julian_date":2460928.85029654,"ra":46.2146231,"range_km":1506.07509695,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":78.09799904,"angle":2.11561162,"azimuth":175.15388279,"date_time":"2025-09-10 08:24:26 UTC","dec":-42.19858201,"julian_date":2460928.85030812,"ra":46.09784759,"range_km":1505.06031291,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":78.36355243,"angle":1.84897622,"azimuth":175.46588667,"date_time":"2025-09-10 08:24:27 UTC","dec":-41.94048201,"julian_date":2460928.85031969,"ra":45.98187273,"range_km":1504.07307903,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":78.62905144,"angle":1.58388665,"azimuth":175.79278995,"date_time":"2025-09-10 08:24:28 UTC","dec":-41.68197148,"julian_date":2460928.85033126,"ra":45.86668972,"range_km":1503.11344864,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":78.89446079,"angle":1.32147564,"azimuth":176.13565481,"date_time":"2025-09-10 08:24:29 UTC","dec":-41.42306003,"julian_date":2460928.85034284,"ra":45.75228987,"range_km":1502.18147366,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":79.15974251,"angle":1.06396985,"azimuth":176.4956444,"date_time":"2025-09-10 08:24:30 UTC","dec":-41.16375733,"julian_date":2460928.85035441,"ra":45.63866461,"range_km":1501.2772046,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":79.42485561,"angle":0.81633124,"azimuth":176.87403477,"date_time":"2025-09-10 08:24:31 UTC","dec":-40.90407307,"julian_date":2460928.85036599,"ra":45.52580546,"range_km":1500.40069052,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":79.68975576,"angle":0.59149984,"azimuth":177.2722285,"date_time":"2025-09-10 08:24:32 UTC","dec":-40.64401702,"julian_date":2460928.85037756,"ra":45.41370407,"range_km":1499.55197906,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":79.9543948,"angle":0.42769036,"azimuth":177.69177026,"date_time":"2025-09-10 08:24:33 UTC","dec":-40.38359898,"julian_date":2460928.85038914,"ra":45.3023522,"range_km":1498.73111636,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":80.21872029,"angle":0.40740648,"azimuth":178.1343647,"date_time":"2025-09-10 08:24:34 UTC","dec":-40.12282879,"julian_date":2460928.85040071,"ra":45.1917417,"range_km":1497.93814711,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":80.48267494,"angle":0.54724632,"azimuth":178.60189688,"date_time":"2025-09-10 08:24:35 UTC","dec":-39.86171635,"julian_date":2460928.85041228,"ra":45.08186455,"range_km":1497.1731145,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":80.74619591,"angle":0.76408043,"azimuth":179.09645591,"date_time":"2025-09-10 08:24:36 UTC","dec":-39.60027159,"julian_date":2460928.85042386,"ra":44.97271281,"range_km":1496.43606024,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":81.00921411,"angle":1.00964655,"azimuth":179.62036207,"date_time":"2025-09-10 08:24:37 UTC","dec":-39.33850448,"julian_date":2460928.85043543,"ra":44.86427866,"range_km":1495.72702451,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":81.27165325,"angle":1.26749591,"azimuth":180.17619827,"date_time":"2025-09-10 08:24:38 UTC","dec":-39.07642502,"julian_date":2460928.850447,"ra":44.75655437,"range_km":1495.04604599,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":81.5334288,"angle":1.53154951,"azimuth":180.76684637,"date_time":"2025-09-10 08:24:39 UTC","dec":-38.81404328,"julian_date":2460928.85045858,"ra":44.64953232,"range_km":1494.39316179,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":81.79444674,"angle":1.7991682,"azimuth":181.39552941,"date_time":"2025-09-10 08:24:40 UTC","dec":-38.55136932,"julian_date":2460928.85047015,"ra":44.54320497,"range_km":1493.76840751,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":82.05460213,"angle":2.06904297,"azimuth":182.06586053,"date_time":"2025-09-10 08:24:41 UTC","dec":-38.28841327,"julian_date":2460928.85048173,"ra":44.4375649,"range_km":1493.17181719,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":82.31377732,"angle":2.34045455,"azimuth":182.78190001,"date_time":"2025-09-10 08:24:42 UTC","dec":-38.02518527,"julian_date":2460928.8504933,"ra":44.33260476,"range_km":1492.6034233,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":82.57183994,"angle":2.61297488,"azimuth":183.54822154,"date_time":"2025-09-10 08:24:43 UTC","dec":-37.7616955,"julian_date":2460928.85050488,"ra":44.22831733,"range_km":1492.06325675,"tle_epoch":"2025-09-09 21:49:21 UTC"},{"altitude":82.82864042,"angle":2.88633237,"azimuth":184.36998945,"date_time":"2025-09-10 08:24:44 UTC","dec":-37.49795417,"julian_date":2460928.85051645,"ra":44.12469544,"range_km":1491.55134686,"tle_epoch":"2025-09-09 21:49:21 UTC"}]},"ONEWEB-0147 (47293)":{"name":"ONEWEB-0147","norad_id":47293,"positions":[{"altitude":77.36457428,"angle":2.72945229,"azimuth":183.93917195,"date_time":"2025-09-10 08:24:46 UTC","dec":-42.95054349,"julian_date":2460928.8505396,"ra":43.65989657,"range_km":1249.1602634,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":77.68950292,"angle":2.42098851,"azimuth":184.08172918,"date_time":"2025-09-10 08:24:47 UTC","dec":-42.62418685,"julian_date":2460928.85055117,"ra":43.6576287,"range_km":1247.83334513,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":78.01497769,"angle":2.11731611,"azimuth":184.23233944,"date_time":"2025-09-10 08:24:48 UTC","dec":-42.29720032,"julian_date":2460928.85056275,"ra":43.65537359,"range_km":1246.54022167,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":78.34097705,"angle":1.82118596,"azimuth":184.39169161,"date_time":"2025-09-10 08:24:49 UTC","dec":-41.96959823,"julian_date":2460928.85057432,"ra":43.65313102,"range_km":1245.28099988,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":78.66747841,"angle":1.53736955,"azimuth":184.56055517,"date_time":"2025-09-10 08:24:50 UTC","dec":-41.64139511,"julian_date":2460928.85058589,"ra":43.65090076,"range_km":1244.05578428,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":78.99445805,"angle":1.27459787,"azimuth":184.7397923,"date_time":"2025-09-10 08:24:51 UTC","dec":-41.31260568,"julian_date":2460928.85059747,"ra":43.64868262,"range_km":1242.86467697,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":79.32189091,"angle":1.04937003,"azimuth":184.9303722,"date_time":"2025-09-10 08:24:52 UTC","dec":-40.98324484,"julian_date":2460928.85060904,"ra":43.64647637,"range_km":1241.7077776,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":79.64975044,"angle":0.89127806,"azimuth":185.13338819,"date_time":"2025-09-10 08:24:53 UTC","dec":-40.65332767,"julian_date":2460928.85062062,"ra":43.64428181,"range_km":1240.58518335,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":79.9780084,"angle":0.83981982,"azimuth":185.35007808,"date_time":"2025-09-10 08:24:54 UTC","dec":-40.32286946,"julian_date":2460928.85063219,"ra":43.64209875,"range_km":1239.49698887,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":80.30663458,"angle":0.91378406,"azimuth":185.5818488,"date_time":"2025-09-10 08:24:55 UTC","dec":-39.99188563,"julian_date":2460928.85064376,"ra":43.63992697,"range_km":1238.44328624,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":80.63559651,"angle":1.08835227,"azimuth":185.83030606,"date_time":"2025-09-10 08:24:56 UTC","dec":-39.6603918,"julian_date":2460928.85065534,"ra":43.63776628,"range_km":1237.42416496,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":80.96485907,"angle":1.32469143,"azimuth":186.09729059,"date_time":"2025-09-10 08:24:57 UTC","dec":-39.32840376,"julian_date":2460928.85066691,"ra":43.6356165,"range_km":1236.43971188,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":81.29438401,"angle":1.59587988,"azimuth":186.38492244,"date_time":"2025-09-10 08:24:58 UTC","dec":-38.99593744,"julian_date":2460928.85067849,"ra":43.63347743,"range_km":1235.49001118,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":81.62412938,"angle":1.88718996,"azimuth":186.6956557,"date_time":"2025-09-10 08:24:59 UTC","dec":-38.66300895,"julian_date":2460928.85069006,"ra":43.63134888,"range_km":1234.57514434,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":81.9540488,"angle":2.1908046,"azimuth":187.03234643,"date_time":"2025-09-10 08:25:00 UTC","dec":-38.32963454,"julian_date":2460928.85070163,"ra":43.62923069,"range_km":1233.69519012,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":82.28409059,"angle":2.50241158,"azimuth":187.39833778,"date_time":"2025-09-10 08:25:01 UTC","dec":-37.99583062,"julian_date":2460928.85071321,"ra":43.62712266,"range_km":1232.85022448,"tle_epoch":"2025-09-10 03:05:15 UTC"},{"altitude":82.61419654,"angle":2.81949888,"azimuth":187.79756746,"date_time":"2025-09-10 08:25:02 UTC","dec":-37.66161374,"julian_date":2460928.85072478,"ra":43.62502463,"range_km":1232.0403206,"tle_epoch":"2025-09-10 03:05:15 UTC"}]},"STARLINK-33622 (63066)":{"name":"STARLINK-33622","norad_id":63066,"positions":[{"altitude":77.3800266,"angle":2.94087878,"azimuth":173.24035174,"date_time":"2025-09-10 08:24:29 UTC","dec":-42.86207073,"julian_date":2460928.85034284,"ra":46.77550952,"range_km":499.42256828,"tle_epoch":"2025-09-10 01:34:42 UTC"}]},"STARLINK-6305 (56394)":{"name":"STARLINK-6305","norad_id":56394,"positions":[{"altitude":79.56346598,"angle":2.81491976,"azimuth":196.13543105,"date_time":"2025-09-10 08:24:48 UTC","dec":-40.32672276,"julian_date":2460928.85056275,"ra":41.05045255,"range_km":572.94279952,"tle_epoch":"2025-09-10 05:26:21 UTC"},{"altitude":80.09471065,"angle":2.24113741,"azimuth":193.41310331,"date_time":"2025-09-10 08:24:49 UTC","dec":-39.95513255,"julian_date":2460928.85057432,"ra":41.85574108,"range_km":572.05795879,"tle_epoch":"2025-09-10 05:26:21 UTC"},{"altitude":80.60299219,"angle":1.77521934,"azimuth":190.38384224,"date_time":"2025-09-10 08:24:50 UTC","dec":-39.5770356,"julian_date":2460928.85058589,"ra":42.65455974,"range_km":571.25532693,"tle_epoch":"2025-09-10 05:26:21 UTC"},{"altitude":81.08377088,"angle":1.52274543,"azimuth":187.01263091,"date_time":"2025-09-10 08:24:51 UTC","dec":-39.19262023,"julian_date":2460928.85059747,"ra":43.44671601,"range_km":570.53525238,"tle_epoch":"2025-09-10 05:26:21 UTC"},{"altitude":81.53177822,"angle":1.59139094,"azimuth":183.26565226,"date_time":"2025-09-10 08:24:52 UTC","dec":-38.80208209,"julian_date":2460928.85060904,"ra":44.23203015,"range_km":569.89804947,"tle_epoch":"2025-09-10 05:26:21 UTC"},{"altitude":81.94100094,"angle":1.94920145,"azimuth":179.11406647,"date_time":"2025-09-10 08:24:53 UTC","dec":-38.40562363,"julian_date":2460928.85062062,"ra":45.01033515,"range_km":569.34399776,"tle_epoch":"2025-09-10 05:26:21 UTC"},{"altitude":82.3047486,"angle":2.47484739,"azimuth":174.53950053,"date_time":"2025-09-10 08:24:54 UTC","dec":-38.00345366,"julian_date":2460928.85063219,"ra":45.78147667,"range_km":568.87334142,"tle_epoch":"2025-09-10 05:26:21 UTC"}]}},"total_position_results":47,"total_satellites":4},"performance":{"jd_times":[2460928.8499377486,2460928.8499493226,2460928.8499608967,2460928.8499724707,2460928.8499840447,2460928.8499956187,2460928.8500071927,2460928.8500187667,2460928.8500303407,2460928.8500419147,2460928.8500534887,2460928.8500650628,2460928.8500766368,2460928.850088211,2460928.850099785,2460928.850111359,2460928.850122933,2460928.850134507,2460928.850146081,2460928.850157655,2460928.850169229,2460928.850180803,2460928.850192377,2460928.850203951,2460928.850215525,2460928.850227099,2460928.850238673,2460928.850250247,2460928.850261821,2460928.850273395,2460928.850284969,2460928.850296543,2460928.850308117,2460928.850319691,2460928.850331265,2460928.850342839,2460928.850354413,2460928.850365987,2460928.850377561,2460928.850389135,2460928.850400709,2460928.850412283,2460928.850423857,2460928.850435431,2460928.850447005,2460928.850458579,2460928.850470153,2460928.850481727,2460928.850493301,2460928.850504875,2460928.850516449,2460928.850528023,2460928.850539597,2460928.850551171,2460928.8505627452,2460928.8505743193,2460928.8505858933,2460928.8505974673,2460928.8506090413,2460928.8506206153,2460928.8506321893,2460928.8506437633,2460928.8506553373,2460928.8506669113,2460928.8506784854,2460928.8506900594,2460928.8507016334,2460928.8507132074,2460928.8507247814,2460928.8507363554,2460928.8507479294,2460928.8507595034,2460928.8507710774,2460928.8507826515,2460928.8507942255,2460928.8508057995,2460928.8508173735,2460928.8508289475,2460928.8508405215,2460928.8508520955,2460928.8508636695,2460928.8508752435,2460928.8508868176,2460928.8508983916,2460928.8509099656,2460928.8509215396,2460928.8509331136,2460928.8509446876,2460928.8509562616,2460928.8509678356],"points_in_fov":47,"propagation_time":3.76,"satellites_processed":27804,"tle_time":1.718,"total_time":5.481},"source":"IAU CPS SatChecker","version":"1.5.0"}

In [ ]:
# Extract RA/Dec positions for each satellite
satellites = {}

for sat_key, sat_data in data['data']['satellites'].items():
    if sat_key not in satellites:
        satellites[sat_key] = []
    # Add ra, dec, and julian_date from each position
    for position in sat_data['positions']:
        satellites[sat_key].append([
            position['ra'],
            position['dec'],
            position['julian_date']
        ])

print(f"Public catalog satellites passing through the specified FOV, as seen from Rubin Observatory, starting at {start_time_jd} JD for {duration} seconds:")
for sat in satellites:
    print(f"{sat}: {len(satellites[sat])} points")

In [ ]:
# Simple WCS for the plot
wcs = WCS(naxis=2)
wcs.wcs.crpix = [0, 0]
wcs.wcs.crval = [ra_center, dec_center]
wcs.wcs.cdelt = [0.1, 0.1]
wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']

fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(1, 1, 1, projection=wcs)
center = SkyCoord(ra=ra_center*u.deg, dec=dec_center*u.deg, frame='icrs')
position_angles = np.linspace(0, 360, 360) * u.deg
circle_points = center.directional_offset_by(position_angles, fov_radius * u.deg)

# Query FOV
ax.plot(circle_points.ra.deg, circle_points.dec.deg, 
        c='0.75', ls='--', label='Query FOV', transform=ax.get_transform('world'))

# Satellite tracks
for sat_name in satellites.keys():
    ra = [pos[0] for pos in satellites[sat_name]]
    dec = [pos[1] for pos in satellites[sat_name]]
    coords = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
    
    # Satellite track
    ax.plot(coords.ra.deg, coords.dec.deg, 
            transform=ax.get_transform('world'),
            label=sat_name)
        
    # Satellite position points
    ax.scatter(coords.ra.deg, coords.dec.deg, 
               transform=ax.get_transform('world'), s=6)
    
    # Start and end labels
    times = [pos[2] for pos in satellites[sat_name]]
    if coords.ra.deg[0] < coords.ra.deg[-1]:
        ax.text(coords.ra.deg[0], coords.dec.deg[0], f'{sat_name}', 
                transform=ax.get_transform('world'), fontsize="6")  
    else:
        ax.text(coords.ra.deg[-1], coords.dec.deg[-1], f'{sat_name}', 
            transform=ax.get_transform('world'), fontsize="6")

ax.set_aspect('equal')
ax.legend(bbox_to_anchor=(0.95, 1), loc='upper left', fontsize="6")
ax.set_xlabel('Right Ascension')
ax.invert_xaxis()
ax.set_ylabel('Declination')
ax.set_title(f'Satellite Passes for {obsinfo} (duration {duration:.1f})')
ax.grid(color='lightgray', linestyle='--')

# Draw the camera on the sky, and label the detectors
for idx, corners in enumerate(cornersList):
    poly = Polygon(
        corners, 
        closed=True, 
        color="#058B8C",
        alpha=0.2,
        transform=ax.get_transform('world')
    )
    try:
        ax.add_patch(poly)
    except AttributeError:  # the corners list is not available
        continue
    else:
        ax.text(
            corners[1][0], corners[2][1], detectors[idx], 
            transform=ax.get_transform("world"),
            fontsize=5,
            color='#313333'
        )
plt.savefig(f"/home/c/cslage/u/Satellites/images/Satellite_Passes_{obsinfo}.png")    
# Option: plot a circle representing the LSSTCam FOV instead
#rubin_fov_radius = 1.761111
#rubin_circle_points = center.directional_offset_by(position_angles, rubin_fov_radius * u.deg)
#ax.plot(rubin_circle_points.ra.deg, rubin_circle_points.dec.deg,
#        c='0.75', ls=':', label='LSSTCam FOV', transform=ax.get_transform('world'))